In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import  StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,LeakyReLU,Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Accuracy
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from concurrent.futures import ThreadPoolExecutor


In [2]:
data=pd.read_csv('../datasets/DE.csv').iloc[:,1:33]
labels=pd.read_csv('Labels.csv').iloc[:,1:4]

In [3]:
data

,Fp1,AF3,F3,F7,FC5,FC1,C3,T7,CP5,CP1,...,FC2,Cz,C4,T8,CP6,CP2,P4,P8,PO4,O2
0,0.357722,0.398458,0.432269,0.368557,0.322225,0.400559,0.419050,0.532254,0.682733,0.390814,...,0.452038,0.387942,0.498521,0.689926,0.624214,0.407988,0.335688,0.442606,0.343261,0.420804
1,0.571534,0.463614,0.329329,0.438190,0.517336,0.374373,0.344395,0.346231,20.708374,34.430216,...,21.273990,25.121916,61.430073,92.334186,46.127887,24.727844,18.603230,60.800346,8.171317,41.365383
2,54.277232,66.102727,57.552890,39.455265,33.789203,40.626313,44.559464,43.311745,61.487298,20.313902,...,4.926175,5.501340,6.345347,3.172822,2.488285,2.774186,2.715449,4.077937,4.544456,5.410853
3,3.097682,5.526222,6.169488,1.913767,4.665268,4.898037,4.739518,5.582462,4.077192,2.903961,...,2.560489,1.934467,54.513456,45.309573,52.145925,141.898953,105.310198,29.206678,54.406616,23.555717
4,35.026478,45.797580,85.974207,110.956061,61.673440,59.417301,72.205146,56.012920,85.383463,30.207533,...,32.208024,125.933292,75.760785,89.144470,57.698608,87.225871,72.259202,56.611330,36.287765,342.425733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,1.725262,1.972093,3.817855,2.135671,1.843789,3.484924,2.253486,1.891672,1.881349,3.121688,...,1.990208,2.786594,2.232472,1.780077,2.259226,1.609347,2.742443,2.486992,1.512607,1.604923
1276,1.705549,1.683243,2.212001,1.974757,1.764401,1.637269,1.663807,1.794125,2.398166,2.563464,...,7.245345,2.976572,4.901484,10.247920,2.933405,2.857161,1.673762,2.347903,2.092766,3.173533
1277,2.670116,4.177936,3.201277,9.497051,2.171290,2.660452,2.299116,2.949663,3.079421,2.563609,...,3.587761,2.508271,2.720883,4.495438,3.433350,3.300418,2.838896,3.629131,3.062503,3.341008
1278,5.155825,3.137196,4.354097,2.732545,2.366251,3.221599,4.318935,2.799928,4.139038,3.283016,...,3.106623,2.654657,6.129860,11.440720,5.133079,2.148946,20.122765,11.727774,14.020321,4.592282


In [4]:
labels

,Valence,Arousal,Dominance
0,7.71,7.60,6.90
1,8.10,7.31,7.28
2,8.58,7.54,9.00
3,4.94,6.01,6.12
4,6.96,3.92,7.19
...,...,...,...
1275,3.91,6.96,5.82
1276,2.81,6.13,6.06
1277,3.05,7.01,5.10
1278,3.99,7.17,4.85


In [5]:
cols=['Fp1','F3','F7','C3','T7','CP5','P3','O1','Fp2','F8','T8']
#cols=data.columns
scaler=StandardScaler()
scaled_data=pd.DataFrame(scaler.fit_transform(data[cols]),columns=cols)

In [6]:
scaled_labels=labels>=5
encoder=LabelEncoder()
encoded_labels=[]
encoded_labels.append(encoder.fit_transform(scaled_labels.iloc[:,0]))# 0 for valence and 1 for arousal
encoded_labels=pd.DataFrame(np.array(encoded_labels).transpose(),columns=['label'])

In [7]:
scaled_data=scaled_data.to_numpy().reshape(1280,11)
encoded_labels = to_categorical(encoded_labels, num_classes=2)

In [9]:
class SpinalNet:

    def __init__(self,shape:tuple,units:list):
        super().__init__()
        self.shape=shape
        self.hl_units=units
        

    def create_model(self,):
        input_layer=[Input(shape=(self.shape[0],),name=f'Input_layer{i+1}')
                     for i in range(len(self.hl_units))]
        hidden_layers=[Dense(self.hl_units[0],activation=LeakyReLU(alpha=0.3))(input_layer[0])]
        for i in range(1,len(self.hl_units)):
            merge=concatenate([input_layer[i],hidden_layers[i-1]])
            hidden_layers.append(Dense(self.hl_units[i],activation=LeakyReLU(alpha=0.3))(merge))
        penultimate=concatenate(hidden_layers)
        output=Dense(self.shape[1],activation='softmax')(penultimate)
        model45=Model(inputs=input_layer,outputs=output)
        return model45

In [10]:
def train_and_evaluate(train_index,test_index,fold):
    X_train, X_test = scaled_data[train_index], scaled_data[test_index]
    y_train, y_test = encoded_labels[train_index], encoded_labels[test_index]
    neurons=[50]*16
    model=SpinalNet((2,2),neurons)
    model=model.create_model()
    model.compile(loss=categorical_crossentropy,optimizer=Adam(learning_rate=0.001),metrics=[Accuracy()])
    lyrs,inp,reps=len(neurons),2,1
    inputs=[X_train[:,(i%lyrs)*inp:((i%lyrs)+1)*inp] for i in range(lyrs*reps)]
    test=[X_test[:,(i%lyrs)*inp:((i%lyrs)+1)*inp] for i in range(lyrs*reps)]
    model.fit(inputs,y_train,epochs=150)
    model.save(f'../model_saves/SpinalNet/DE/model_valence/model_thread_{fold}.keras')
    result=model.predict(test)
    new_result=[]
    for i in (result):
        new_result.append(np.where(i==i.max())[0])
    new_result=np.array(new_result).flatten()
    new_y=[]
    for i in (y_test):
        new_y.append(np.where(i==i.max())[0])
    new_y=np.array(new_y).flatten()
    accuracy = accuracy_score(new_y,new_result)
    return accuracy

In [11]:
k_fold = KFold(n_splits=32,shuffle=False)
with ThreadPoolExecutor() as executor:
    futures = []
    fold=0
    for train_index, test_index in k_fold.split(scaled_data):
        fold+=1
        futures.append(executor.submit(train_and_evaluate, train_index, test_index, fold))
    results = [future.result() for future in futures]

print("Cross-validation accuracies:", results)
print("Mean accuracy:", np.mean(results))


C:\Users\indra\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
39/39 ━━━━━━━━━━━━━━━━━━━━ 35s 96ms/step - accuracy: 0.0000e+00 - loss: 0.6918
14/39 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.0000e+00 - loss: 0.6933Epoch 2/150
39/39 ━━━━━━━━━━━━━━━━━━━━ 35s 104ms/step - accuracy: 0.0000e+00 - loss: 0.6811
11/39 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.0000e+00 - loss: 0.7169Epoch 2/150
39/39 ━━━━━━━━━━━━━━━━━━━━ 37s 60ms/step - accuracy: 0.0000e+00 - loss: 0.6851
23/39 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.0000e+00 - loss: 0.6978Epoch 2/150
39/39 ━━━━━━━━━━━━━━━━━━━━ 37s 60ms/step - accuracy: 0.0000e+00 - loss: 0.6964
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.0000e+00 - loss: 0.7203Epoch 2/150
39/39 ━━━━━━━━━━━━━━━━━━━━ 37s 80ms/step - accuracy: 0.0000e+00 - loss: 0.7197
33/39 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.0000e+00 - loss: 0.6506Epoch 2/150
39/39 ━━━━━━━━━

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=SpinalNet_DE1/model_valence/model_thread_1.